<a href="https://colab.research.google.com/github/cmlieu/ProjetoFinal_IA_Google/blob/main/Pre%C3%A7os.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install google-genai
!pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

CHATBOT

In [3]:
from google import genai
from google.genai import types
from googleapiclient.discovery import build # Importe a biblioteca para usar a API do Google Search
import os

# Importar chave API
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

#criar variável de ambiente
os.environ ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

#instalar um cliente que irá receber as informações do Gemini
client = genai.Client()

# Inicialização e configuração
modeloIA = "gemini-2.0-flash"
#modeloIA = "gemini-exp-1206" #nao adiantou mudar o modelo

#criar objeto chat

chat = client.chats.create(model = modeloIA)

#criar uma persona para resposta
chat_config = types.GenerateContentConfig(
  system_instruction = "você é um assistente pessoal, que sempre responde de forma sucinta.",
)

chat = client.chats.create(model = modeloIA, config = chat_config)


In [4]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [5]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [13]:
# --- Agente 1: Buscador de preços no Google --- #


def agente_buscador(preco, cidade, data_de_hoje):

    buscador = Agent(
       name = "agente_buscador",
       model = "gemini-2.0-flash",
       instruction = """

            Você é um assistente de pesquisa cuja tarefa é buscar preços no Google Search para os seguintes produtos [PRODUTOS DEFINIDOS PELO USUÁRIO]
      no local [LOCAL DEFINIDO PELO USUÁRIO].

        **ENTRADA DO USUÁRIO:** O usuário fornecerá uma lista de um ou mais produtos e o local (cidade, bairro, etc.).

        **ESTRATÉGIA DE BUSCA DINÂMICA E CRITÉRIOS DE SELEÇÃO:**

        1.  **Preparação da Busca:** Para cada produto na lista fornecida pelo usuário, formule termos de busca relevantes para encontrar
        onde ele é vendido e qual o seu preço na cidade especificada. Exemplos de termos de busca para um produto chamado "[NOME DO PRODUTO]" seriam:
            * "preço [NOME DO PRODUTO] [CIDADE]"
            * "onde comprar [NOME DO PRODUTO] [CIDADE]"
            * "[NOME DO PRODUTO] comprar online [CIDADE]"
            * "[NOME DE GRANDES SUPERMERCADOS/ATACADOS ONLINE na cidade] [NOME DO PRODUTO]"
        2.  **Explorar Diversidade de Lojas:** Busque os preços dos produtos dos supermercados, atacados, mercados mais populares
        (i.e., mais reconhecidos com base na quantidade de visitas ao site). Procure apenas lojas que existem na cidade especificada e que tenham site com link.
        3.  **Navegação e Verificação Rigorosa (Por Produto):** Para cada site encontrado:
            * Para cada produto da lista do usuário, navegue até a página específica desse produto no site. Use a busca interna do site, se necessário.
            * **VERIFIQUE ATENTAMENTE se o *valor numérico do preço* está claramente indicado em cada página E se há um *link direto para a página desse produto*.**
            * **SE O PREÇO OU O LINK de QUALQUER UM dos produtos da lista NÃO forem encontrados, DESCARTE IMEDIATAMENTE ESTE SITE POR COMPLETO.**
        4.  **Datas Recentes:** Priorize os preços em até 30 dias da data da consulta do usuário.
        5.  **Retorno Apenas de Empresas Completas:** Retorne os dados (nome da loja, preços e links) APENAS para as empresas que apresentarem o
        valor numérico do preço E o link direto para a página de TODOS os produtos listados pelo usuário. Não combine dados de lojas diferentes.
        6.  IMPORTANTE: **Número Alvo de Empresas:** Continue a busca até identificar pelo menos 3 empresas que atendam a todos os critérios.

        **Instrução de Saída:**

        Para cada uma das empresas encontradas que possuem o preço E o link de *todos* os produtos fornecidos pelo usuário, apresente:

        * **Nome da Loja:** [Nome da Loja]
        * **Preços e Links:**
            * [Nome do Produto 1]: [Valor do Preço] - [Link do Produto 1]
            * [Nome do Produto 2]: [Valor do Preço] - [Link do Produto 2]
            * ... (para todos os produtos da lista do usuário)

        **IMPORTANTE:** Garanta que a saída esteja claramente associada a cada produto específico da lista do usuário, incluindo o preço e o link correspondente.


       """,
       tools = [google_search]
    )

    entrada_do_agente_buscador = f"Preço: {preco}\nCidade: {cidade}\nData de hoje: {data_de_hoje}"

# Executa o agente
    noticias = call_agent(buscador, entrada_do_agente_buscador)
    return noticias


In [1]:
# --- Agente 2: Analisador de preços --- #

def agente_analisador(preco, cidade, noticias_buscadas):

    analisador = Agent(
       name = "agente_analisador",
       model = "gemini-2.0-flash",
       instruction = """

       Você é um analisador de preços, cuja tarefa é receber os dados de preços e links de diferentes empresas fornecidos pelo Agente Buscador e determinar qual empresa oferece o menor preço total para todos os produtos.

        1.  Analise cuidadosamente os dados fornecidos pelo Agente Buscador para cada empresa. Confira se há link para a loja listada pelo Agente Buscador. Caso não haja, descarte a empresa.
        2.  Verifique se cada empresa listada realmente apresenta um *valor numérico* para o preço e o *link* de *todos* os produtos solicitados.
        Ignore quaisquer empresas que não possuam o preço ou o link de algum dos produtos. Faça a soma dos valores numéricos de todos os produtos para cada empresa.
        3.  Para cada empresa válida, some o valor numérico dos preços de todos os produtos, lembrando de tratar a vírgula como separador decimal, caso ela apareça nos dados de entrada.
        4.  Compare os totais de todas as empresas válidas e retorne apenas os dados da empresa que possui o menor valor total, construindo a resposta da seguinte forma **EXATA**:

        "**Menor Preço Total Encontrado:**" + "\n" +
        "**Empresa:** " + [Nome da Empresa"com o menor preço total] + "\n" +
        "**Preço Total:** R$ " + [Valor total formatado com duas casas decimais, usando vírgula como separador decimal, ex: 123,45] + "\n\n" +
        "**Preços e Links por Produto:**" +"\n" +
        "- " + [Nome do Produto 1] + ": R$ " + [Valor do Preço 1 formatado, usando vírgula como separador decimal, ex: 10,00] + " - [Link da Página do Produto 1]" + "\n" +
        "- " + [Nome do Produto 2] + ": R$ " + [Valor do Preço 2 formatado, usando vírgula como separador decimal, ex: 20,50] + " - [Link da Página do Produto 2]" + "\n" +
        "... (para todos os produtos da lista, seguindo o mesmo padrão '- [Nome do Produto]: R$ [Valor] - [Link]')" + "\n"

        **IMPORTANTE SOBRE A FORMATAÇÃO E LIMPEZA DE TEXTO:**

        * Garanta que haja um espaço **EXATAMENTE** entre cada palavra e símbolo (exceto entre o símbolo 'R$' e o número).
        * Remova quaisquer caracteres especiais, acentos estranhos, símbolos não padrão ou artefatos de formatação.
        * Utilize apenas letras padrão do alfabeto português, números, espaços, vírgulas (como separador decimal), pontos e os símbolos 'R$', ':' e '-'.
        * Revise a ortografia e use pontuação adequada. NÃO junte palavras.
        * Se não houver nenhuma empresa que tenha todos os produtos, retorne os preços dos produtos de cada empresa

        **Exemplo de Saída Esperada:**

        ```
        **Menor Preço Total Encontrado:**
        **Empresa:** Supermercado Bom Preço
        **Preço Total:** R$ 35,99

        **Preços e Links por Produto:**
        - Sal Refinado Cisne 1Kg: R$ 2,89 - [https://www.supermercadobompreco.com.br/sal-refinado-cisne-1kg](https://www.supermercadobompreco.com.br/sal-refinado-cisne-1kg)
        - Café 500g: R$ 33,10 - [https://www.supermercadobompreco.com.br/cafe-torrado-e-moido-500g](https://www.supermercadobompreco.com.br/cafe-torrado-e-moido-500g)
        ```

       """,
       tools = [google_search]
    )

    entrada_do_agente_analisador = f"Preço: {preco}\nCidade: {cidade}\nNotícias buscadas]: {noticias_buscadas}"

# Executa o agente
    analise = call_agent(analisador, entrada_do_agente_analisador)
    return analise

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

def main():
    print("Bem-vindo ao Comparador de Preços (Baseado em IA)!")
    print("Exemplo para listar os produtos? 1 kg de sal, 500 g de café (se necessário, inclua o tipo do produto e a marca")

    while True:
        preco = input("\nQuer saber o menor preço aproximado de quais produtos? (ou 'fim' para sair): ")

        if preco.lower() == "fim":
            print("Obrigado por usar o Comparador de Preços!")
            break
        else:
            cidade = input("Em que local quer saber o preço (bairro ou cidade)? ") # Pede o local
            print("Verificando...") # Indica que a busca está em andamento pela IA
            noticias_buscadas = agente_buscador(preco, cidade, data_de_hoje)
            #print("\n-- Resultado da busca no Google --- \n")
            #display (to_markdown(noticias_buscadas))
            precos_analisados = agente_analisador(preco, cidade, noticias_buscadas)
            print("\n-- Resultado da análise de preços considerando o último mês --- \n")
            display (to_markdown(precos_analisados))
            print ("Nota: Valores estimados. Os preços dependem do estoque, promoções, etc.")

if __name__ == "__main__":
      main()


Bem-vindo ao Comparador de Preços (Baseado em IA)!
Exemplo para listar os produtos? 1 kg de sal, 500 g de café (se necessário, inclua o tipo do produto e a marca

Quer saber o menor preço aproximado de quais produtos? (ou 'fim' para sair): 1 kg de arroz, 500 g de macarrão Barilla, 500 g de café Melitta, 1 kg de açúcar União
Em que local quer saber o preço (bairro ou cidade)? Curitiba
Verificando...

-- Resultado da análise de preços --- 



> Como nenhuma das empresas possui o preço e link de todos os produtos solicitados, apresento os preços dos produtos encontrados em cada empresa:
> 
> **Armazém da Maria:**
> - Arroz Parboilizado Tipo 1 Buriti Pacote 1kg: R$ 5,65 - [https://www.armazemdamaria.com.br/arroz-parboilizado-tipo-1-buriti-pacote-1kg/p](https://www.armazemdamaria.com.br/arroz-parboilizado-tipo-1-buriti-pacote-1kg/p)
> - Açúcar Refinado União 1Kg: R$ 5,28 - [https://www.armazemdamaria.com.br/acucar-refinado-uniao-1kg/p](https://www.armazemdamaria.com.br/acucar-refinado-uniao-1kg/p)
> - Café Melitta Sabor da Fazenda Tradicional 500g: Consulte o site. - [https://www.armazemdamaria.com.br/cafe-melitta-sabor-da-fazenda-tradicional-500g/p](https://www.armazemdamaria.com.br/cafe-melitta-sabor-da-fazenda-tradicional-500g/p)
> - Macarrão Barilla 500g: Não encontrado.
> 
> **Super Muffato Delivery:**
> - Açúcar Refinado União 1kg: R$ 4,80 - [https://www.supermuffato.com.br/produto/acucar-refinado-uniao-1kg/12030](https://www.supermuffato.com.br/produto/acucar-refinado-uniao-1kg/12030)
> - Macarrão Spaghetti Barilla nº5 Grano Duro 500g: R$ 13,40 - [https://www.supermuffato.com.br/produto/macarrao-spaghetti-barilla-n5-grano-duro-500g/11692](https://www.supermuffato.com.br/produto/macarrao-spaghetti-barilla-n5-grano-duro-500g/11692)
> - Café Melitta Tradicional a Vácuo 500g: Consulte o site. - [https://www.supermuffato.com.br/produto/cafe-melitta-tradicional-a-vacuo-500g/12018](https://www.supermuffato.com.br/produto/cafe-melitta-tradicional-a-vacuo-500g/12018)
> - Arroz 1kg: Não encontrado.
> 
> **Belíssima Casa di Frutas:**
> - AÇÚCAR REFINADO LINHA CLÁSSICOS UNIÃO 1KG: R$ 7,98 - [https://www.belissimacasadifrutas.com.br/acucar-refinado-linha-classicos-uniao-1kg](https://www.belissimacasadifrutas.com.br/acucar-refinado-linha-classicos-uniao-1kg)
> - Arroz 1kg: Não encontrado.
> - Macarrão Barilla 500g: Não encontrado.
> - Café Melitta 500g: Não encontrado.
> 


Nota: Valores estimados. Os preços dependem do estoque, promoções, etc.

Quer saber o menor preço aproximado de quais produtos? (ou 'fim' para sair): fim
Obrigado por usar o Comparador de Preços!


In [ ]:
!pip cache purge

Files removed: 132
